**Linear regression with elastic net penalty**


The goal of the following code is to compute the bias, variance and MSE of using LO to estimate the out-out-of-error (OO) as a function of $p$ (and $n = \delta p$). The theory claims that
$$
MSE \sim var \sim O(1/n)=O(1/p).
$$

To do so we sample synthetic data and numerically regress $\log(MSE)$ (and $\log$(var)  against $\log p$ and report the $R^2$ and the scaling exponent, eg. $ MSE \sim C/n^a$ or $\log MSE = \log C + a \log n$.

The $\lambda$ used in the code scales like $1/n$ as $n,p$ changes because the equation optimized in the `ElasticNetCV` function when fitting the model is:

$$
\min_{\beta} \left\{ \frac{1}{2n} \sum_{i=1}^n (y_i - x_i^\top \beta)^2 + \lambda \left( \alpha \|\beta\|_1 + \frac{1}{2} (1 - \alpha) \|\beta\|^2_2 \right) \right\}
$$

where $ \lambda $ is the regularization parameter, $ \alpha $ is the `l1_ratio` mixing parameter between L1 and L2 penalties, $ n $ is the number of samples, $ y_i $ are the observed values, $ x_i $ are the predictors, and $ \beta$ are the coefficients to be estimated.



The reported $df$ is to make sure that the selected $\lambda$ is reasonable, in the mid-range giving us a number of estimated coefficients that we expect.




In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import RidgeCV
from scipy.stats import laplace, norm
import statsmodels.api as sm
import matplotlib.pyplot as plt


# Set the seed for reproducibility
np.random.seed(1)

# Parameters
#Haolin: I changed the array to equal space in log scale
logp_       = np.arange(0,6)
plogstep    = 10**(0.2)
p_          = 120* plogstep**(logp_)
p_          = p_.astype(int)
delta       = 1.5 # n/p
rho         = 0.2 # k/p
alpha_elnet = 1 # l1_ratio, alpha_elnet=1 is L1,  alpha_elnet=0 is L2
MCMCsamples = 200
lo          = np.zeros((MCMCsamples, len(p_)))
outErr      = np.zeros((MCMCsamples, len(p_)))
MSE         = np.zeros(len(p_))
bias2       = np.zeros(len(p_))
var         = np.zeros(len(p_))
o           = 1
a           = 0
R2          = 0
pos         = False

# Generate data and fit model
for i, p in enumerate(p_):
    #if(i < 5):
    #    continue
    n = int(p * delta)
    k = int(p * rho)
    for s in range(MCMCsamples):
        #if(s < 21):
        #    continue
        if(i == 5 and s%10 == 0):
            print(s)
        beta_star = np.zeros(p)
        beta_star[0:k] = 1 #laplace.rvs(scale=1/np.sqrt(2), size=k) * np.sqrt(n/k)
        X = np.random.normal(0, 1/np.sqrt(n), (n, p))
        y = X @ beta_star + norm.rvs(scale=o, size=n)
        if alpha_elnet > 0:
            lambdaS = np.array([1 / n])
            model = ElasticNetCV(l1_ratio=alpha_elnet, tol=0.001, max_iter=10000, positive = pos, alphas=lambdaS, cv=n, fit_intercept=False).fit(X, y)
            lo[s, i] = np.mean(model.mse_path_) / 2  # Adjust as needed;
        else:
            lambdaS = np.array([100 / n])
            model = RidgeCV(alphas=lambdaS, store_cv_values=True, tol=0.001, max_iter=10000,  cv=None, fit_intercept=False).fit(X, y)
            lo[s, i] = np.mean(model.cv_values_) / 2  # Adjust as needed;

        beta_hat = model.coef_
        outErr[s, i] = (o**2 + np.sum((beta_star - beta_hat) ** 2) / n) / 2
        #np.count_nonzero(beta_hat)
        df = np.mean(np.abs(beta_hat))/np.mean(np.abs(beta_star))
        # Calculate bias and MSE
        bias2[i] = (np.mean(outErr[:s+1, i] -lo[:s+1, i])) ** 2
        MSE[i] = np.mean((lo[:s+1, i] - outErr[:s+1, i]) ** 2)
        var[i] = MSE[i] - bias2[i]
        if i > 0:
            ls_fit = sm.OLS(np.log(MSE[:i+1]), sm.add_constant(np.log(p_[:i+1]))).fit()
            a = ls_fit.params[1]
            R2 = ls_fit.rsquared
        if s % 100 == 99:  # Print every 100 iterations of s
            print(f"p= {p:4d}| s={s+1:3d}| df={df:.2f}, lmb={lambdaS[0]*n:.1f}| lo={lo[s, i]:.2f}| outErr={outErr[s, i]:.2f}| MSE ={MSE[i]:.3f} | scaling = {a:.2f} | R2_MSE = {R2:.2f}")


ls_mse_fit = sm.OLS(np.log(MSE), sm.add_constant(np.log(p_))).fit()
a_mse = ls_mse_fit.params[1]
R2_mse = ls_mse_fit.rsquared

ls_var_fit = sm.OLS(np.log(var), sm.add_constant(np.log(p_))).fit()
a_var = ls_var_fit.params[1]
R2_var = ls_var_fit.rsquared

ls_bias2_fit = sm.OLS(np.log(bias2), sm.add_constant(np.log(p_))).fit()
a_bias2 = ls_bias2_fit.params[1]
R2_bias2 = ls_bias2_fit.rsquared

print(f" MSE: scaling = {a_mse:.2f} | R2 = {R2_mse:.2f}")
print(f" var: scaling = {a_var:.2f} | R2 = {R2_var:.2f}")
print(f" bias2: scaling = {a_bias2:.2f} | R2 = {R2_bias2:.2f}")


# Calculate MSE standard error
MSE_SE = np.std((lo - outErr) ** 2, axis=0) / np.sqrt(MCMCsamples)
print("Mean squared errors:", np.mean((lo - outErr) ** 2, axis=0))
print("MSE standard errors:", MSE_SE)

results = np.column_stack((lo, outErr))

np.savetxt("linear-noncons/lasso/delta1_5p120.csv", results, delimiter=",")

C:\Users\arnab\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\arnab\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\arnab\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


p=  120| s=100| df=0.80, lmb=1.0| lo=0.59| outErr=0.59| MSE =0.006 | scaling = 0.00 | R2_MSE = 0.00
p=  120| s=200| df=1.02, lmb=1.0| lo=0.73| outErr=0.59| MSE =0.006 | scaling = 0.00 | R2_MSE = 0.00
p=  190| s=100| df=1.17, lmb=1.0| lo=0.68| outErr=0.58| MSE =0.003 | scaling = -1.55 | R2_MSE = 1.00
p=  190| s=200| df=0.84, lmb=1.0| lo=0.52| outErr=0.58| MSE =0.003 | scaling = -1.29 | R2_MSE = 1.00
p=  301| s=100| df=1.03, lmb=1.0| lo=0.62| outErr=0.59| MSE =0.003 | scaling = -0.93 | R2_MSE = 0.95
p=  301| s=200| df=1.16, lmb=1.0| lo=0.69| outErr=0.60| MSE =0.002 | scaling = -1.15 | R2_MSE = 0.99
p=  477| s=100| df=0.87, lmb=1.0| lo=0.63| outErr=0.58| MSE =0.001 | scaling = -1.20 | R2_MSE = 1.00
p=  477| s=200| df=0.81, lmb=1.0| lo=0.55| outErr=0.59| MSE =0.001 | scaling = -1.11 | R2_MSE = 1.00
p=  757| s=100| df=0.80, lmb=1.0| lo=0.56| outErr=0.58| MSE =0.001 | scaling = -1.08 | R2_MSE = 1.00
p=  757| s=200| df=0.79, lmb=1.0| lo=0.59| outErr=0.58| MSE =0.001 | scaling = -1.04 | R2_MSE

In [ ]:
#np.savetxt("linear-noncons/elnet/lo_delta0_5p200.csv", lo, delimiter=",")
#np.savetxt("linear-noncons/elnet/outErr_delta0_5p200.csv", outErr, delimiter=",")

In [ ]:
ls_mse_fit = sm.OLS(np.log(MSE), sm.add_constant(np.log(p_))).fit()
a_mse = ls_mse_fit.params[1]
R2_mse = ls_mse_fit.rsquared

ls_var_fit = sm.OLS(np.log(var), sm.add_constant(np.log(p_))).fit()
a_var = ls_var_fit.params[1]
R2_var = ls_var_fit.rsquared

ls_bias2_fit = sm.OLS(np.log(bias2), sm.add_constant(np.log(p_))).fit()
a_bias2 = ls_bias2_fit.params[1]
R2_bias2 = ls_bias2_fit.rsquared

print(f" MSE: scaling = {a_mse:.2f} | R2 = {R2_mse:.2f}")
print(f" var: scaling = {a_var:.2f} | R2 = {R2_var:.2f}")
print(f" bias2: scaling = {a_bias2:.2f} | R2 = {R2_bias2:.2f}")


# Calculate MSE standard error
MSE_SE = np.std((lo - outErr) ** 2, axis=0) / np.sqrt(MCMCsamples)
print("Mean squared errors:", np.mean((lo - outErr) ** 2, axis=0))
print("MSE standard errors:", MSE_SE)

results = np.column_stack((lo, outErr))



In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

ls_mse_fit = sm.OLS(np.log(MSE), sm.add_constant(np.log(p_))).fit()
a_mse = ls_mse_fit.params[1]
R2_mse = ls_mse_fit.rsquared

ls_var_fit = sm.OLS(np.log(var), sm.add_constant(np.log(p_))).fit()
a_var = ls_var_fit.params[1]
R2_var = ls_var_fit.rsquared

ls_bias2_fit = sm.OLS(np.log(bias2), sm.add_constant(np.log(p_))).fit()
a_bias2 = ls_bias2_fit.params[1]
R2_bias2 = ls_bias2_fit.rsquared

print(f" MSE: scaling = {a_mse:.2f} | R2 = {R2_mse:.2f}")
print(f" var: scaling = {a_var:.2f} | R2 = {R2_var:.2f}")
print(f" bias2: scaling = {a_bias2:.2f} | R2 = {R2_bias2:.2f}")


# Calculate MSE standard error
MSE_SE = np.std((lo - outErr) ** 2, axis=0) / np.sqrt(MCMCsamples)
print("Mean squared errors:", np.mean((lo - outErr) ** 2, axis=0))
print("MSE standard errors:", MSE_SE)

# Calculate logarithms
log_bias2 = np.log(bias2)
log_p_ = np.log(p_)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(p_, bias2, color='blue')
plt.xlabel('p_')
plt.ylabel('bias^2')
plt.title('bias^2 vs p_')
plt.grid(True)
plt.show()


In [ ]:
#Added by Haolin: side by side boxplot for two groups of results, e.g. with and without constraints
def side_by_side_boxplot(lo_mat1, outErr_mat1,lo_mat2, outErr_mat2, x_labels):
    '''
    Creates a side-by-side boxplot for log squared error against log p.
    lo_mat1: num_sim by num_p matrix, consisting of lo in group 1
    outErr_mat1: num_sim by num_p matrix, consisting of out of sample error in group 1
    lo_mat2: lo of group 2
    outErr_mat2: outErr of group 2
    x_labels: list of str, x labels
    '''
    num_sim = len(lo_mat1)
    num_p = len(lo_mat1[0])

    error_sq_mat1 = (lo_mat1 - outErr_mat1) ** 2
    log_err_sq_mat1 = np.log(error_sq_mat1)
    error_sq_mat2 = (lo_mat2 - outErr_mat2) ** 2
    log_err_sq_mat2 = np.log(error_sq_mat2)

    MSE1 = np.mean(error_sq_mat1, axis=0)
    log_MSE1 = np.log(MSE1)
    MSE2 = np.mean(error_sq_mat2, axis=0)
    log_MSE2 = np.log(MSE2)

    # Create a figure and axis object
    fig, ax = plt.subplots()

    # Create boxplots for each age group
    ax.boxplot(log_err_sq_mat1, positions=np.arange(num_p)+0.8, widths=0.3)
    ax.boxplot(log_err_sq_mat2, positions=np.arange(num_p)+1.2, widths=0.3)

    # Set labels and title
    ax.set_xticks(np.arange(1, num_p+1))
    ax.set_xticklabels(x_labels)
    ax.set_xlabel('Log p')
    ax.set_ylabel('Log squared error between LO and Err_out')
    #ax.set_title('Absolute Error against P (log scale)')

    # Fit a linear regression model
    x_p = np.arange(1, num_p+1).reshape(-1, 1)
    model1 = LinearRegression().fit(x_p, log_MSE1)
    model2 = LinearRegression().fit(x_p, log_MSE2)

    # Plot regression line for MSE against log(p)
    regression_line1 = model1.predict(x_p)
    ax.plot(np.arange(num_p)+0.8, regression_line1, color='red', label='Group 1')
    regression_line2 = model2.predict(x_p)
    ax.plot(np.arange(num_p)+1.2, regression_line2, color='blue', label='Group 2')
    print('group 1 slope:', model1.coef_, 'group 2 slope:', model2.coef_)

    #Scatterplot of the MSEs
    ax.scatter(np.arange(num_p)+0.8, log_MSE1, marker = 'd', color = 'red')
    ax.scatter(np.arange(num_p)+1.2, log_MSE2, marker = 'd', color = 'blue')

    # Show the legend
    ax.legend()

    # Show the plot
    plt.grid(True)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import pandas as pd 
from sklearn.linear_model import LinearRegression

lasso_noncons = pd.read_csv('lin_elnet_nocons.csv',header = None)
lasso_poscons = pd.read_csv('lin_elnet_poscons.csv', header = None)

lomat_lasso_noncons = lasso_noncons.iloc[np.arange(0,241),np.arange(0,5)].to_numpy()
outErrmat_lasso_noncons = lasso_noncons.iloc[np.arange(0,241),np.arange(5,10)].to_numpy()

lomat_lasso_poscons = lasso_poscons.iloc[np.arange(0,241),np.arange(0,5)].to_numpy()
outErrmat_lasso_poscons = lasso_poscons.iloc[np.arange(0,241),np.arange(5,10)].to_numpy()

x_labels= ['1','2','3','4','5']

lo_mat1 = lomat_lasso_noncons
outErr_mat1 = outErrmat_lasso_noncons

lo_mat2 = lomat_lasso_poscons
outErr_mat2 = outErrmat_lasso_poscons

num_sim = len(lo_mat1)
num_p = len(lo_mat1[0])

error_sq_mat1 = (lo_mat1 - outErr_mat1) ** 2
log_err_sq_mat1 = np.log(error_sq_mat1)
error_sq_mat2 = (lo_mat2 - outErr_mat2) ** 2
log_err_sq_mat2 = np.log(error_sq_mat2)

MSE1 = np.mean(error_sq_mat1, axis=0)
log_MSE1 = np.log(MSE1)
MSE2 = np.mean(error_sq_mat2, axis=0)
log_MSE2 = np.log(MSE2)

fig, ax = plt.subplots()

# Create boxplots for each age group
ax.boxplot(log_err_sq_mat1, positions=np.arange(num_p)+0.8, widths=0.3)
ax.boxplot(log_err_sq_mat2, positions=np.arange(num_p)+1.2, widths=0.3)

# Set labels and title
ax.set_xticks(np.arange(1, num_p+1))
ax.set_xticklabels(x_labels)
ax.set_xlabel('Log p')
ax.set_ylabel('Log squared error between LO and Err_out')
#ax.set_title('Absolute Error against P (log scale)')

# Fit a linear regression model
x_p = np.arange(1, num_p+1).reshape(-1, 1)
model1 = LinearRegression().fit(np.log(p_), log_MSE1)
model2 = LinearRegression().fit(np.log(p_), log_MSE2)

# Plot regression line for MSE against log(p)
regression_line1 = model1.predict(x_p)
ax.plot(np.arange(num_p)+0.8, regression_line1, color='red', label='No constraint')
regression_line2 = model2.predict(x_p)
ax.plot(np.arange(num_p)+1.2, regression_line2, color='blue', label='Positive constraint')
print('group 1 slope:', model1.coef_, 'group 2 slope:', model2.coef_)

#Scatterplot of the MSEs
ax.scatter(np.arange(num_p)+0.8, log_MSE1, marker = 'd', color = 'red')
ax.scatter(np.arange(num_p)+1.2, log_MSE2, marker = 'd', color = 'blue')

# Show the legend
ax.legend()

# Show the plot
plt.grid(True)
plt.show()

In [ ]:
np.shape(lo_mat1)

In [ ]:
MSE1